# Table of Contents
 <p><div class="lev1"><a href="#Measure-Dynamic-Functional-Connectivity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Measure Dynamic Functional Connectivity</a></div><div class="lev2"><a href="#Initialize-Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2"><a href="#Load-CoreData-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load CoreData</a></div><div class="lev2"><a href="#Compute-Functional-Connectivity-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Compute Functional Connectivity</a></div><div class="lev3"><a href="#Process-Navon-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Process Navon</a></div><div class="lev3"><a href="#Process-Stroop-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Process Stroop</a></div>

# Measure Dynamic Functional Connectivity

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    
except:
    print 'NOT IPYTHON'

from __future__ import division
from IPython.display import display

import os
import sys
import glob
import json

import numpy as np
import pandas as pd
import scipy.io as io
import h5py
import matplotlib.pyplot as plt

os.chdir('../')
import Codebase
conv_adj_matr_to_cfg_matr = Codebase.Networks.configuration.convert_adj_matr_to_cfg_matr
conv_cfg_vec_to_adj_matr = Codebase.Networks.configuration.convert_conn_vec_to_adj_matr
os.chdir('./Analysis_Notebooks/')

import scipy.stats as stats

path_Remotes = '/Users/akhambhati/Remotes'
path_CoreData = path_Remotes + '/CORE.fMRI_cogcontrol.medaglia'
path_PeriphData = path_Remotes + '/RSRCH.NMF_CogControl'
path_ExpData = path_PeriphData + '/e01-FuncNetw'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Load CoreData

In [ ]:
df_navon = io.loadmat('{}/NavonBlockedSeriesScale125.mat'.format(path_CoreData), struct_as_record=False)
df_stroop = io.loadmat('{}/StroopBlockedSeriesScale125.mat'.format(path_CoreData), struct_as_record=False)

n_subj = 28
n_fix_block = 12
n_tsk_block = 6
n_roi = 262

## Compute Functional Connectivity

### Process Navon

In [ ]:
for subj_id in xrange(n_subj):
    proc_item = '{}/Subject_{}.Navon'.format(path_ExpData, subj_id)

    adj_rs_pos = np.zeros((n_tsk_block, n_roi, n_roi))
    adj_rs_neg = np.zeros((n_tsk_block, n_roi, n_roi))    
    adj_lo_pos = np.zeros((n_tsk_block, n_roi, n_roi))
    adj_lo_neg = np.zeros((n_tsk_block, n_roi, n_roi))    
    adj_hi_pos = np.zeros((n_tsk_block, n_roi, n_roi))
    adj_hi_neg = np.zeros((n_tsk_block, n_roi, n_roi))    

    cnt = 0
    for fix_block in xrange(0, n_fix_block, 2):
        data = np.vstack((np.array(df_navon['data'][subj_id][fix_block+1].NFix, dtype='f'),
                          np.array(df_navon['data'][subj_id][fix_block+2].NFix, dtype='f'))).T        
        adj = np.corrcoef(data)
        adj_rs_pos[cnt, :, :] = stats.threshold(adj, 0)
        adj_rs_neg[cnt, :, :] = stats.threshold(-1*adj, 0)        

        cnt += 1

    for tsk_block in xrange(n_tsk_block):
        data = np.array(df_navon['data'][subj_id][tsk_block].NS, dtype='f').T
        adj = np.corrcoef(data)
        adj_lo_pos[tsk_block, :, :] = stats.threshold(adj, 0)
        adj_lo_neg[tsk_block, :, :] = stats.threshold(-1*adj, 0)        

        data = np.array(df_navon['data'][subj_id][tsk_block].S, dtype='f').T
        adj = np.corrcoef(data)
        adj_hi_pos[tsk_block, :, :] = stats.threshold(adj, 0)
        adj_hi_neg[tsk_block, :, :] = stats.threshold(-1*adj, 0)        

    np.savez(proc_item,
             adj_rs_pos = adj_rs_pos,
             adj_rs_neg = adj_rs_neg,
             adj_lo_pos = adj_lo_pos,
             adj_lo_neg = adj_lo_neg,
             adj_hi_pos = adj_hi_pos,
             adj_hi_neg = adj_hi_neg)

### Process Stroop

In [ ]:
for subj_id in xrange(n_subj):
    proc_item = '{}/Subject_{}.Stroop'.format(path_ExpData, subj_id)
    
    adj_rs_pos = np.zeros((n_tsk_block, n_roi, n_roi))
    adj_rs_neg = np.zeros((n_tsk_block, n_roi, n_roi))    
    adj_lo_pos = np.zeros((n_tsk_block, n_roi, n_roi))
    adj_lo_neg = np.zeros((n_tsk_block, n_roi, n_roi))    
    adj_hi_pos = np.zeros((n_tsk_block, n_roi, n_roi))
    adj_hi_neg = np.zeros((n_tsk_block, n_roi, n_roi))    
    
    cnt = 0
    for fix_block in xrange(0, n_fix_block, 2):
        data = np.vstack((np.array(df_stroop['data'][subj_id][fix_block+1].SFix, dtype='f'),
                          np.array(df_stroop['data'][subj_id][fix_block+2].SFix, dtype='f'))).T
        adj = np.corrcoef(data)
        adj_rs_pos[cnt, :, :] = stats.threshold(adj, 0)
        adj_rs_neg[cnt, :, :] = stats.threshold(-1*adj, 0)        
        
        cnt += 1
        
    for tsk_block in xrange(n_tsk_block):
        data = np.array(df_stroop['data'][subj_id][tsk_block].IE, dtype='f').T        
        adj = np.corrcoef(data)
        adj_lo_pos[tsk_block, :, :] = stats.threshold(adj, 0)
        adj_lo_neg[tsk_block, :, :] = stats.threshold(-1*adj, 0)        
        
        data = np.array(df_stroop['data'][subj_id][tsk_block].E, dtype='f').T                
        adj = np.corrcoef(data)
        adj_hi_pos[tsk_block, :, :] = stats.threshold(adj, 0)
        adj_hi_neg[tsk_block, :, :] = stats.threshold(-1*adj, 0)        

    np.savez(proc_item,
             adj_rs_pos = adj_rs_pos,
             adj_rs_neg = adj_rs_neg,
             adj_lo_pos = adj_lo_pos,
             adj_lo_neg = adj_lo_neg,
             adj_hi_pos = adj_hi_pos,
             adj_hi_neg = adj_hi_neg)

In [ ]:
%matplotlib inline

tt = 5
plt.figure(figsize=(8,12))
ax = plt.subplot(1,1,1); 
ax.hist(adj.reshape(-1), 100)
ax.set_yscale('log')

In [ ]:
%matplotlib inline
proc_items = np.random.permutation(glob.glob('{}/Subject_*.npz'.format(path_ExpData)))[0]
adj = np.load(proc_items)
tt = np.random.randint(6)

adj1 = adj['adj_rs_pos'][tt, ...]; adj1[np.isnan(adj1)] = 0;
adj2 = adj['adj_rs_neg'][tt, ...]; adj2[np.isnan(adj2)] = 0;
adj3 = adj['adj_lo_pos'][tt, ...]; adj3[np.isnan(adj3)] = 0;
adj4 = adj['adj_lo_neg'][tt, ...]; adj4[np.isnan(adj4)] = 0;
adj5 = adj['adj_hi_pos'][tt, ...]; adj5[np.isnan(adj5)] = 0;
adj6 = adj['adj_hi_neg'][tt, ...]; adj6[np.isnan(adj6)] = 0;

cmap = 'viridis'

plt.figure(figsize=(8,12))
ax = plt.subplot(3,2,1); mat = ax.matshow(adj1, cmap=cmap, vmin=0, vmax=1); plt.colorbar(mat, ax=ax)
ax = plt.subplot(3,2,2); mat = ax.matshow(adj2, cmap=cmap, vmin=0, vmax=1); plt.colorbar(mat, ax=ax)
ax = plt.subplot(3,2,3); mat = ax.matshow(adj3, cmap=cmap, vmin=0, vmax=1); plt.colorbar(mat, ax=ax)
ax = plt.subplot(3,2,4); mat = ax.matshow(adj4, cmap=cmap, vmin=0, vmax=1); plt.colorbar(mat, ax=ax)
ax = plt.subplot(3,2,5); mat = ax.matshow(adj5, cmap=cmap, vmin=0, vmax=1); plt.colorbar(mat, ax=ax)
ax = plt.subplot(3,2,6); mat = ax.matshow(adj6, cmap=cmap, vmin=0, vmax=1); plt.colorbar(mat, ax=ax)

plt.figure()
ax = plt.subplot(1,1,1);
mat = ax.matshow(adj1, cmap=cmap, vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)
ax.set_axis_off()
plt.savefig('./e01-Figures/Adj_RS_Pos.svg')

plt.figure()
ax = plt.subplot(1,1,1);
mat = ax.matshow(adj2, cmap=cmap, vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)
ax.set_axis_off()
plt.savefig('./e01-Figures/Adj_RS_Neg.svg')

plt.figure()
ax = plt.subplot(1,1,1);
mat = ax.matshow(adj3, cmap=cmap, vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)
ax.set_axis_off()
plt.savefig('./e01-Figures/Adj_LO_Pos.svg')

plt.figure()
ax = plt.subplot(1,1,1);
mat = ax.matshow(adj4, cmap=cmap, vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)
ax.set_axis_off()
plt.savefig('./e01-Figures/Adj_LO_Neg.svg')

plt.figure()
ax = plt.subplot(1,1,1);
mat = ax.matshow(adj5, cmap=cmap, vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)
ax.set_axis_off()
plt.savefig('./e01-Figures/Adj_HI_Pos.svg')

plt.figure()
ax = plt.subplot(1,1,1);
mat = ax.matshow(adj6, cmap=cmap, vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)
ax.set_axis_off()
plt.savefig('./e01-Figures/Adj_HI_Neg.svg')

In [ ]:
cfg = np.load('/Users/akhambhati/Remotes/RSRCH.NMF_CogControl/e02b-FuncSubg/NMF_Optimization.CfgMatr.npz')
cfg_matr = cfg['cfg_matr'].T
cfg_matr[np.isnan(cfg_matr)] = 0

plt.figure(figsize=(16,12))
ax = plt.subplot(111)
mat = ax.matshow(cfg_matr, 
                 aspect=cfg_matr.shape[1]/cfg_matr.shape[0],
                 cmap=cmap)
ax.set_axis_off()
plt.savefig('./e01-Figures/Cfg_Matr.svg')

In [ ]:
%matplotlib inline
cmap = 'viridis'

print('Plotting Subgraph Factors...')

df = np.load('/Users/akhambhati/Remotes/RSRCH.NMF_CogControl/e02-FuncSubg/NMF_Consensus.npz')
fac_subnet = df['fac_subnet'].T
fac_subnet[np.isnan(fac_subnet)] = 0
fac_coef = df['fac_coef']
n_fac = fac_coef.shape[0]
n_obs = fac_coef.shape[1]
arr_obs = np.arange(n_obs)

plt.figure(figsize=(16,12))
ax = plt.subplot(111)
mat = ax.matshow(fac_subnet, 
                 aspect=fac_subnet.shape[1]/fac_subnet.shape[0],
                 cmap=cmap)
ax.set_axis_off()
plt.savefig('./e01-Figures/Fac_Subnet.svg')
plt.close()

plt.figure(figsize=(16,12))
ax = plt.subplot(111)
mat = ax.matshow(conv_cfg_vec_to_adj_matr(fac_subnet[:, 1]), cmap=cmap)
ax.set_axis_off()
plt.savefig('./e01-Figures/Fac_Subnet_Adj.svg')
plt.close()


print('Plotting Subgraph Coefficients...')
plt.figure(figsize=(16,12))
ax = plt.subplot(111)
for i in xrange(15):
    norm_coef = fac_coef[i, :]
    norm_coef -= norm_coef.mean()
    norm_coef /= norm_coef.max()
    
    ax.plot(arr_obs[:72*3], norm_coef[:72*3]+i)

# Axis Settings
ax.xaxis.set_ticks_position('bottom')
ax.set_xlim([0, 72*3])
ax.set_xlabel('Time Windows')
ax.yaxis.set_ticks_position('left')
ax.set_ylim([-1, 15])
ax.set_ylabel('Functional Subgraphs')

plt.savefig('./e01-Figures/Fac_Coefs.svg')
plt.close()  

In [ ]:
fac_subnet.shape